In [0]:
# Imports
from pyspark.sql import functions as F, types as T
from pyspark.sql import DataFrame
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from typing import List, Tuple
from statsmodels.tsa.arima.model import ARIMA

In [0]:
path = "/Volumes/steam_project/steam_raw_data/raw_data/games.csv"

df_steam = (
    spark.read
    .option("header", "true")
    .option("multiLine", "true")
    .option("quote", '"')
    .option("escape", '"')
    .option("ignoreLeadingWhiteSpace", "true")
    .option("ignoreTrailingWhiteSpace", "true")
    .csv(path)
)
display(df_steam)


Unfortunately, we see the shift in the original data: Discount DLC count contains unreliable data, while its true values are stored in the neighbouring column on the right. The pattern continues even further till the end.

It can be explained by the flaw in original transformation to csv, as json analog of the file does not contain the data current Discount DLC count column has. 

With this, we need to transform our data for further work


In [0]:
from pyspark.sql import functions as F

cols = df_steam.columns
i = cols.index("DiscountDLC count")

new_cols = []

# Columns before the shift must stay the same (ID, name, relese date, etc)
for c in cols[:i]:
    new_cols.append(F.col(c).alias(c))

# Shift: now column[i] = values from column[i+1]
for j in range(i, len(cols)-1):
    new_cols.append(F.col(cols[j+1]).alias(cols[j]))

# Last column gets NULL because nothing shifts into it
new_cols.append(F.lit(None).alias(cols[-1]))

df_steam_fixed = df_steam.select(*new_cols)
drop_col = df_steam_fixed.columns[-1]
df_steam_fixed = df_steam_fixed.drop(drop_col)
display(df_steam_fixed)


Let's make aditional changes: transform numerical values from string, change tags, categories and genres to arrays for ferther use 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType
from pyspark.sql.types import ArrayType, StringType

# Transforming columns to float
cols_to_float = ["Peak CCU", "Required age", "DiscountDLC count", "Metacritic score", "User score", "Positive", "Negative", "Score rank", "Achievements", "Recommendations", "Average playtime forever", "Average playtime two weeks", "Median playtime forever", "Median playtime two weeks"]

for c in cols_to_float:
    df_steam_fixed = df_steam_fixed.withColumn(c, F.round(F.col(c).cast("float"),2))

# Separate case for Price, because we want it rounded as it was in original
df_steam_fixed = df_steam_fixed.withColumn(
    "Price", F.col("Price").cast(DecimalType(10, 2))
)

# transforming string format to arrays
cols_to_array = ["Categories", "Genres", "Tags"]
for c in cols_to_array:
    df_steam_fixed = df_steam_fixed.withColumn(c, F.split(F.col(c), ","))


# Separate case for languages that have pseudo arrays 
cols_to_ar = ["Supported languages", "Full audio languages"]

for c in cols_to_ar:
    df_steam_fixed = (
        df_steam_fixed
        .withColumn(c, F.regexp_replace(F.col(c), "'", '"'))
        .withColumn(c, F.from_json(F.col(c), ArrayType(StringType())))
    )

display(df_steam_fixed)


Still, we need to look into Release dates and their format

In [0]:
from pyspark.sql import functions as F

df_steam_fixed.select(
    F.min("Release date").alias("earliest_release"),
    F.max("Release date").alias("latest_release")
).show()


We see that currently the latest release date is listed as Sep 9 ,2024, wgich is false, as we know that data has released games from 2025

In [0]:
row_df = df_steam_fixed.filter(F.col("AppID") == "3183790")
display(row_df)


This means we also need to reformat the release data.

We know from visually exploring the data, that there are at least 2 formats of dates: MMM d, yyyy and MMM YYYY. In cases where we can find MMM YYYY format, we decided to add date 15th of the month as the day and unify format

In [0]:
from pyspark.sql import functions as F

col = F.col("Release date")

df_steam_fixed = df_steam_fixed.withColumn(
    "Release date",
    F.when(
        # Case 1: full date like Apr 10, 2023
        col.rlike(r"^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}$"),
        F.to_date(col, "MMM d, yyyy")
    ).when(
        # Case 2: month + year like May 2020
        col.rlike(r"^[A-Za-z]{3} [0-9]{4}$"),
        F.date_add(F.to_date(col, "MMM yyyy"), 14)
    )
)


Let's chek the result

In [0]:
display(df_steam_fixed)

In [0]:
from pyspark.sql import functions as F

df_steam_fixed.select(
    F.min("Release date").alias("earliest_release"),
    F.max("Release date").alias("latest_release")
).show()


Now we see true result on release dates and can get accurate statistics

# Missing values exploration

Now as we have more or less reliable data, let's look into the missing values

In [0]:
def is_missing_expr(col_name, dtype, include_empty_str):
    c = F.col(col_name)
    base_null = c.isNull()
    if isinstance(dtype, (T.FloatType, T.DoubleType)):
        cond = base_null | F.isnan(c)
    elif isinstance(dtype, T.StringType):
        cond = base_null | (F.length(F.trim(c)) == 0 if include_empty_str else F.lit(False))
    else:
        cond = base_null
    return F.when(cond, 1).otherwise(0)

def missing_summary(df, columns=None, include_empty_str=True):
    if columns is None:
        columns = df.columns
    columns = [c for c in columns if c in df.columns]
    if not columns:
        raise ValueError("No valid columns were given")

    total_rows = df.count()
    schema_map = {f.name: f.dataType for f in df.schema.fields}

    exprs = [F.sum(is_missing_expr(c, schema_map[c], include_empty_str)).alias(c) for c in columns]
    null_counts_row = df.select(*exprs).collect()[0].asDict()

    rows = []
    for c in columns:
        nulls = int(null_counts_row[c])
        present = total_rows - nulls
        null_pct = (nulls / total_rows * 100.0) if total_rows else 0.0
        rows.append({"column": c, "present_count": present, "null_count": nulls, "null_pct": round(null_pct, 4)})

    return pd.DataFrame(rows).sort_values("null_pct", ascending=False, ignore_index=True)

def missing_matrix(df, columns, max_rows = 5000, include_empty_str = True):
    if columns is None:
        columns = df.columns
    columns = [c for c in columns if c in df.columns]
    if not columns:
        raise ValueError("No valid columns given")

    with_id = df.select(F.monotonically_increasing_id().alias("_rid"), *columns).orderBy("_rid").limit(max_rows)

    schema_map = {f.name: f.dataType for f in df.schema.fields}
    miss_cols = [is_missing_expr(c, schema_map[c], include_empty_str).alias(c) for c in columns]

    bin_df = with_id.select("_rid", *miss_cols).orderBy("_rid").drop("_rid")
    pdf = bin_df.toPandas()
    return pdf

def visualize_missing(df, columns, max_rows = 2000, include_empty_str = True):
    """
    Visualisation that gives basic understanding of amount of pressent and missing data (with % of missing data), plots heatmap showing where missing data is (displays only columns where there are missing values) and bar chart of null % in the data 
    """
    summary_pdf = missing_summary(df, columns=columns, include_empty_str=include_empty_str)
    # Keeping columns that have some missing values
    cols_with_nulls = summary_pdf.loc[summary_pdf["null_count"] > 0, "column"].tolist()

    if not cols_with_nulls:
        print("No missing values detected")
        display(spark.createDataFrame(summary_pdf))
        return summary_pdf

    # Ploting heatmap: green - data is present, red - missing data (null value)
    mat_pdf = missing_matrix(df, columns=cols_with_nulls, max_rows=max_rows, include_empty_str=include_empty_str)
    if not mat_pdf.empty:
        mat = mat_pdf.values.astype(np.int8)
        fig1, ax1 = plt.subplots(figsize=(12, 10))
        cmap = ListedColormap(["#1F449C", "#F05039"])
        norm = BoundaryNorm([-0.5, 0.5, 1.5], cmap.N)
        ax1.imshow(mat, aspect="auto", interpolation="nearest", cmap=cmap, norm=norm)
        ax1.set_title(f"Missingness heatmap")
        ax1.set_xlabel("Columns")
        ax1.set_ylabel("Row number")
        ax1.set_xticks(range(len(mat_pdf.columns)))
        ax1.set_xticklabels(mat_pdf.columns, rotation=90)
        plt.tight_layout()
        plt.show()

    # Ploting bar charts with null % per column
    filtered_summary = summary_pdf[summary_pdf["column"].isin(cols_with_nulls)]
    if not filtered_summary.empty:
        fig2, ax2 = plt.subplots(figsize=(12, 8))
        ax2.bar(filtered_summary["column"], filtered_summary["null_pct"])
        ax2.set_title("Null % per column")
        ax2.set_xlabel("Column")
        ax2.set_ylabel("Null percentage (%)")
        ax2.set_xticklabels(filtered_summary["column"], rotation=90)
        plt.tight_layout()
        plt.show()

    display(spark.createDataFrame(summary_pdf))
    return summary_pdf

In [0]:
summary = visualize_missing(df_steam_fixed, columns=None, max_rows=1500000, include_empty_str=True)

In [0]:
def missing_rate_expr(col: str):
    return (F.count(F.when(F.col(col).isNull(), 1)) / F.count(F.lit(1))).alias(col)

def missing_indicator(col: str):
    return F.when(F.col(col).isNull(), 1).otherwise(0).alias(col + "_miss")

def to_pandas(df, limit=None):
    return (df.limit(limit) if isinstance(limit, int) else df).toPandas()

SAMPLE_FRAC = 0.20  
SEED = 42

In [0]:
null_counts = df_steam_fixed.select([
    F.count(F.when(F.col(c).isNull(), 1)).alias(c) for c in df_steam_fixed.columns
]).collect()[0].asDict()

cols_with_nulls = [c for c, n in null_counts.items() if n and n > 0]

miss_df = df_steam_fixed.select([missing_indicator(c) for c in cols_with_nulls])
miss_df_sample = miss_df.sample(withReplacement=False, fraction=SAMPLE_FRAC, seed=SEED)

pdf = miss_df_sample.toPandas()

corr = pdf.corr(numeric_only=True)
corr.index.name = "col"
corr.columns.name = "col"


In [0]:
N = 40
miss_rates = miss_df_sample.agg(*[F.mean(c).alias(c) for c in miss_df_sample.columns]).collect()[0].asDict()
top_cols = [k for k,_ in sorted(miss_rates.items(), key=lambda kv: kv[1], reverse=True)[:N]]

corr_top = corr.loc[top_cols, top_cols].to_numpy()

plt.figure(figsize=(8,7))
im = plt.imshow(corr_top, aspect='auto')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(ticks=np.arange(len(top_cols)), labels=top_cols, rotation=90)
plt.yticks(ticks=np.arange(len(top_cols)), labels=top_cols)
plt.title("Co-missingness Correlation (Top N columns)")
plt.tight_layout()
plt.show()


## Hidden null values

It is quite common for datasets to have so-called by or team hidden null values: values that cannot be identified as empty by the system, but in relity don't hold any value. 

As hidden nulls we concider empty lists, Strings like "none", "N/A", ? etc.

Let's look into it 

In [0]:
from pyspark.sql import functions as F

cols_to_check = [
    "Supported languages",
    "Full audio languages",
    "Categories",
    "Genres",
    "Tags",
]

conditions = {}

for c in cols_to_check:
    non_empty_filtered = F.expr(f"filter(`{c}`, x -> trim(x) != '')")

    conditions[c] = (
        F.col(c).isNull()
        | (F.size(F.col(c)) == 0)
        | (F.size(non_empty_filtered) == 0)
    )

combined_condition = None
for cond in conditions.values():
    combined_condition = cond if combined_condition is None else (combined_condition | cond)

filtered_df = df_steam_fixed.filter(combined_condition).select(*cols_to_check)

display(filtered_df)


As we can see, many games don't have audio translation, which is still considered as not empty field. 

The missingness can be explained by a simple fact that not all games have dialogues, so there is no need for audio adaptation

Still, in the furhter transformation we will have to work with this empty lists and change them to **null**

Let's move to other cases

In [0]:
from pyspark.sql import functions as F
from functools import reduce
from operator import or_

# Paterns we are looking for
null_patterns = [
    r"^$",                 # empty string
    r"^\s+$",              # whitespace
    r"(?i)^null$",         # null / NULL / Null
    r"(?i)^none$",         # none / NONE
    r"(?i)^n/a$",          # n/a / N/A
    r"(?i)^na$",           # na / NA
    r"^\?$",               # ?
]

string_cols = [c for c, t in df_steam_fixed.dtypes if t == "string"]

print("String columns being checked:")
print(string_cols)

conditions = []

for c in string_cols:
    for pattern in null_patterns:
        conditions.append(F.col(c).rlike(pattern))

if not conditions:
    print("No string columns found to check.")
else:
    combined_condition = reduce(or_, conditions)
    df_null_like = df_steam_fixed.filter(combined_condition)

    print("Number of rows with null-like values in at least one column:", df_null_like.count())

    display(df_null_like)

    print("\nPer-column counts of null-like values:")
    for c in string_cols:
        col_condition = reduce(or_, [F.col(c).rlike(p) for p in null_patterns])
        cnt = df_steam_fixed.filter(col_condition).count()
        if cnt > 0:
            print(f"{c}: {cnt}")


As we can see, there are some more hidden null values in data that we will have to clean in the future

# Exploring numbers

Since we now have more-or-less clear understanding of our data and it's limitations, let's explore information that we can gain from it

In [0]:
from pyspark.sql import functions as F

df_genres = df_steam_fixed.select(F.explode("Genres").alias("Genre"))

df_genre_counts = (
    df_genres.groupBy("Genre")
    .count()
    .orderBy(F.col("count").desc())
    .limit(5)
)

display(df_genre_counts)


In [0]:
top5_pd = df_genre_counts.toPandas()   # convert to pandas for matplotlib
import matplotlib.pyplot as plt

genres = top5_pd["Genre"]
counts = top5_pd["count"]


plt.figure(figsize=(10, 6))

bars = plt.barh(genres, counts)

for bar in bars:
    width = bar.get_width()
    plt.text(
        width + max(counts) * 0.02,
        bar.get_y() + bar.get_height() / 2,
        f"{int(width)}",
        va='center'
    )

plt.xlabel("Number of games")
plt.ylabel("Genres")
plt.title("Top 5 Steam Genres (by number of games)")
plt.tight_layout()
plt.xlim(0, max(counts) * 1.1)
plt.show()

In [0]:
df_cat = df_steam_fixed.select(F.explode("Categories").alias("Category"))

df_cat_counts = (
    df_cat.groupBy("Category")
    .count()
    .orderBy(F.col("count").desc())
    .limit(5)
)

display(df_cat_counts)
top5_cat = df_cat_counts.toPandas() 
category = top5_cat["Category"]
counts = top5_cat["count"]


plt.figure(figsize=(10, 6))

bars = plt.barh(category, counts)

for bar in bars:
    width = bar.get_width()
    plt.text(
        width + max(counts) * 0.02,
        bar.get_y() + bar.get_height() / 2,
        f"{int(width)}",
        va='center'
    )

plt.xlabel("Number of games")
plt.ylabel("Category")
plt.title("Top 5 Steam Categories (by number of games)")
plt.tight_layout()
plt.xlim(0, max(counts) * 1.1)
plt.show()

In [0]:
from pyspark.sql import functions as F
df_year_counts = (
    df_steam_fixed
        .withColumn("year", F.year("Release date"))
        .groupBy("year")
        .count()
        .orderBy("year")
)

df_year_counts_pd = df_year_counts.toPandas()
df_year_counts_pd
years = df_year_counts_pd["year"]
counts = df_year_counts_pd["count"]

plt.figure(figsize=(12, 6))
bars = plt.bar(years, counts)

# Add labels above each bar
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2,   # center the label
        height,                            # place above the bar
        str(int(height)),                  # label value
        ha='center', 
        va='bottom',
        fontsize=10
    )

plt.xlabel("Year")
plt.ylabel("Number of games released")
plt.title("Game releases per year")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 1. Year + exploded genres
df_genre_year = (
    df_steam_fixed
    .withColumn("year", F.year("Release date"))
    .select("year", F.explode("Genres").alias("genre"))
    .filter(F.col("year").isNotNull() & F.col("genre").isNotNull())
)

# 2. Count games per (year, genre)
df_genre_counts = (
    df_genre_year
    .groupBy("year", "genre")
    .count()
)

# 3. For each year, keep only the genre with max count
w = Window.partitionBy("year").orderBy(F.col("count").desc())

df_top_genre_per_year = (
    df_genre_counts
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .drop("rn")
    .orderBy("year")
)

top_genre_pd = df_top_genre_per_year.toPandas()
years = top_genre_pd["year"]
counts = top_genre_pd["count"]
genres = top_genre_pd["genre"]

plt.figure(figsize=(14, 7))
bars = plt.bar(years, counts)

# Add genre labels above each bar
for bar, genre in zip(bars, genres):
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2,
        height,
        genre,
        ha='center',
        va='bottom',
        fontsize=10,
        rotation=45
    )

plt.xlabel("Year")
plt.ylabel("Number of Games (for top genre)")
plt.title("Most Popular Game Genre per Year (by Number of Releases)")
plt.xticks(rotation=45)
plt.ylim(0, max(counts) + 1500)
plt.tight_layout()
plt.show()



In [0]:
from pyspark.sql import functions as F

target_genres = ["Action", "Indie", "Casual"]

# Explode genres + extract year
df_genre_year = (
    df_steam_fixed
        .withColumn("year", F.year("Release date"))
        .select("year", F.explode("Genres").alias("genre"))
        .filter(
            (F.col("year") >= 2010) & (F.col("year") < 2025) &
            F.col("genre").isin(target_genres)
        )
)

# Count games per (year, genre)
df_counts = (
    df_genre_year
        .groupBy("year", "genre")
        .count()
        .orderBy("year", "genre")
)

df_counts_pd = df_counts.toPandas()
df_pivot = df_counts_pd.pivot(index="year", columns="genre", values="count").fillna(0)
plt.figure(figsize=(14, 7))

plt.plot(df_pivot.index, df_pivot["Action"], marker="o", label="Action")
plt.plot(df_pivot.index, df_pivot["Indie"], marker="o", label="Indie")
plt.plot(df_pivot.index, df_pivot["Casual"], marker="o", label="Casual")

plt.xlabel("Year")
plt.ylabel("Number of games")
plt.title("Number of Action, Indie, and Casual games released (2010-2024)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [0]:
from pyspark.sql import functions as F

df_quarter = (
    df_steam_fixed
        .withColumn("quarter", F.quarter("Release date"))
        .groupBy("quarter")
        .count()
        .orderBy("quarter")
        .toPandas()
)

quarters = df_quarter["quarter"].astype(str)
counts = df_quarter["count"]

plt.figure(figsize=(10, 6))
bars = plt.bar(quarters, counts)

# Add labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        str(int(height)),
        ha="center",
        va="bottom",
        fontsize=9
    )

plt.xlabel("Quarter")
plt.ylabel("Number of Games Released")
plt.title("Distribution of Game Releases by Quarter")
plt.tight_layout()
plt.show()


In [0]:
from pyspark.sql import functions as F

total_games = df_steam_fixed.count()

df_platform_support = (
    df_steam_fixed.agg(
        (F.sum(F.when(F.col("Windows") == "True", 1).otherwise(0)) / total_games * 100).alias("Windows"),
        (F.sum(F.when(F.col("Mac") == "True", 1).otherwise(0)) / total_games * 100).alias("Mac"),
        (F.sum(F.when(F.col("Linux") == "True", 1).otherwise(0)) / total_games * 100).alias("Linux")
    )
).toPandas().T

df_platform_support.columns = ["percentage"]
import matplotlib.pyplot as plt

platforms = df_platform_support.index
percentages = df_platform_support["percentage"]

plt.figure(figsize=(8, 6))
bars = plt.bar(platforms, percentages, color=["#4c72b0", "#55a868", "#c44e52"])

# Add % labels
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2,
        height + 1,
        f"{height:.1f}%",
        ha='center',
        va='bottom',
        fontsize=10
    )

plt.ylabel("Games %")
plt.title("Platform Support: Windows, Mac, Linux")
plt.ylim(0, max(percentages) + 10)
plt.tight_layout()
plt.show()



## Exploring games by prices 

In [0]:
from pyspark.sql import functions as F

df_price_counts = (
    df_steam_fixed
        .withColumn("is_free", F.when(F.col("Price") == 0, "Free").otherwise("Paid"))
        .groupBy("is_free")
        .count()
)

df_price_counts_pd = df_price_counts.toPandas()

labels = df_price_counts_pd["is_free"]
sizes = df_price_counts_pd["count"]

plt.figure(figsize=(7, 7))
plt.pie(
    sizes,
    labels=labels,
    autopct="%1.1f%%",
    startangle=140
)

plt.title("Distribution of Free vs Paid games")
plt.axis("equal")
plt.show()



In [0]:
import matplotlib.pyplot as plt
import numpy as np
df_year_fp = (
    df_steam_fixed
        .withColumn("year", F.year("Release date"))
        .withColumn("type", F.when(F.col("Price") == 0, "Free").otherwise("Paid"))
        .groupBy("year", "type")
        .count()
)

df_year_fp_pd = (
    df_year_fp
        .groupBy("year")
        .pivot("type", ["Free", "Paid"])
        .sum("count")
        .fillna(0)
        .orderBy("year")
        .toPandas()
)
df_year_fp_pd["total"] = df_year_fp_pd["Free"] + df_year_fp_pd["Paid"]
df_year_fp_pd["free_pct"] = df_year_fp_pd["Free"] / df_year_fp_pd["total"] * 100
df_year_fp_pd["paid_pct"] = df_year_fp_pd["Paid"] / df_year_fp_pd["total"] * 100
years = df_year_fp_pd["year"]
free_pct = df_year_fp_pd["free_pct"]
paid_pct = df_year_fp_pd["paid_pct"]

x = np.arange(len(years)) * 1.3

plt.figure(figsize=(18, 7))

bars_free = plt.bar(x, free_pct, label="Free (%)")
bars_paid = plt.bar(x, paid_pct, bottom=free_pct, label="Paid (%)")

plt.xticks(x, years, rotation=45)
plt.xlabel("Year")
plt.ylabel("Percentage of Games")
plt.title("Share of Free-to-Play vs Paid Games Over Time")
plt.legend()
plt.tight_layout()
plt.show()


In [0]:
# Most expensive games/apps in Steam
df_most_expencive = (
    df_steam_fixed
        .select("Name", "Price", "Genres", "Estimated owners", "Release date")
        .orderBy(F.col("Price").desc())
        .limit(6)
)
display(df_most_expencive)

In [0]:
from pyspark.sql import functions as F

df_price_year = (
    df_steam_fixed
        .withColumn("year", F.year("Release date"))
        .groupBy("year")
        .agg(F.avg("Price").alias("avg_price"))
        .orderBy("year")
)

df_price_year_pd = df_price_year.toPandas()

years = df_price_year_pd["year"]
avg_prices = df_price_year_pd["avg_price"]

plt.figure(figsize=(12, 6))
plt.plot(years, avg_prices, marker='o')

# Add labels on each point
for x, y in zip(years, avg_prices):
    plt.text(
        x, y, 
        f"{y:.2f}", 
        ha='center', 
        va='bottom',
        fontsize=10
    )

plt.xlabel("Year")
plt.ylabel("Average price (USD)")
plt.title("Average game prices per years")
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [0]:
from statsmodels.tsa.arima.model import ARIMA

price_series = df_price_year_pd.set_index("year")["avg_price"].astype(float) # from average price per year plot

# Fit ARIMA model
model = ARIMA(price_series, order=(1, 1, 1))
results = model.fit()

# Forecast next 5 years
n_forecast = 5
forecast_res = results.get_forecast(steps=n_forecast)
forecast_mean = forecast_res.predicted_mean

last_year = int(price_series.index.max())
future_years = list(range(last_year + 1, last_year + 1 + n_forecast))

forecast_mean.index = future_years

plt.figure(figsize=(15, 6))

# Historical
plt.plot(price_series.index, price_series.values, marker="o", label="Historical avg price")

# Forecast
plt.plot(forecast_mean.index, forecast_mean.values, marker="o", linestyle="--", label="Forecast avg price")

for x, y in zip(price_series.index, price_series.values):
    plt.text(
        x, y+0.3, 
        f"{y:.2f}", 
        ha='center', 
        va='bottom',
        fontsize=8
    )
for x, y in zip(forecast_mean.index, forecast_mean.values):
    plt.text(
        x, y+0.3, 
        f"{y:.2f}", 
        ha='center', 
        va='bottom',
        fontsize=8
    )
plt.xlabel("Year")
plt.ylabel("Average price (USD)")
plt.title("Average game price over time using ARIMA forecast")
plt.grid(True, linestyle="--", alpha=0.5)
plt.xticks(list(price_series.index) + future_years, rotation=45)
plt.ylim(0, max(price_series.values.max(), forecast_mean.values.max())+1)
plt.legend()
plt.tight_layout()
plt.show()

## Exploring games by popularuty (estimated number of owners)

In [0]:
display(
    df_steam_fixed
        .select("Estimated owners")
        .distinct()
        .orderBy("Estimated owners")
)


In [0]:
owner_bins = [
    "0 - 0",
    "0 - 20000",
    "20000 - 50000",
    "50000 - 100000",
    "100000 - 200000",
    "200000 - 500000",
    "500000 - 1000000",
    "1000000 - 2000000",
    "2000000 - 5000000",
    "5000000 - 10000000",
    "10000000 - 20000000",
    "20000000 - 50000000",
    "50000000 - 100000000",
    "100000000 - 200000000",
]

df_owners = (
    df_steam_fixed
        .groupBy("Estimated owners")
        .count()
)

owners_pd = df_owners.toPandas()
cat_type = pd.CategoricalDtype(categories=owner_bins, ordered=True)
owners_pd["Estimated owners"] = owners_pd["Estimated owners"].astype(cat_type)

owners_pd = owners_pd.sort_values("Estimated owners")
owners_pd = owners_pd.dropna(subset=["Estimated owners"])

plt.figure(figsize=(12, 6))
plt.bar(owners_pd["Estimated owners"].astype(str), owners_pd["count"])

plt.xlabel("Estimated number of owners")
plt.ylabel("Number of games")
plt.title("Distribution of games by estimated number of owners")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [0]:
df_groups = (
    df_steam_fixed
        .withColumn(
            "Popularity group",
            F.when(F.col("Estimated owners") == "0 - 0", "Unplayed")
             .when(F.col("Estimated owners").isin("0 - 20000", "20000 - 50000"), "Niche")
             .when(F.col("Estimated owners").isin(
                    "50000 - 100000",
                    "100000 - 200000",
                    "200000 - 500000"
                  ), "Mid-scale")
             .when(F.col("Estimated owners").isin(
                    "500000 - 1000000",
                    "1000000 - 2000000",
                    "2000000 - 5000000",
                    "5000000 - 10000000",
                    "10000000 - 20000000",
                    "20000000 - 50000000",
                    "50000000 - 100000000",
                    "100000000 - 200000000"
                  ), "Hit")
             .otherwise(None)
        )
        .filter(F.col("Popularity group").isNotNull())
)

df_group_counts = (
    df_groups
        .groupBy("Popularity group")
        .count()
        .toPandas()
)


order = ["Unplayed", "Niche", "Mid-scale", "Hit"]
df_group_counts = df_group_counts.set_index("Popularity group").reindex(order).reset_index()

groups = df_group_counts["Popularity group"]
counts = df_group_counts["count"]

plt.figure(figsize=(8, 6))
bars = plt.bar(groups, counts)

# add labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        str(int(height)),
        ha="center",
        va="bottom",
        fontsize=10
    )

plt.xlabel("Popularity group (by estimated owners)")
plt.ylabel("Number of games")
plt.title("Distribution of games by popularity")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()



## Let's look into games

In [0]:
df_top_owners = (
    df_steam_fixed
        .filter(F.col("Estimated owners") == "100000000 - 200000000")
        .select("Name", "Estimated owners", "Release date")
        .orderBy(F.col("Release date").desc())
        .limit(15)
)

display(df_top_owners)


As we can see, there is only one game in Steam that crossed 100000000 users and it is Dota2

Now, let's explore games with a bit smaller number of users

In [0]:
df_top_owners = (
    df_steam_fixed
        .filter(F.col("Estimated owners") == "50000000 - 100000000")
        .select("Name", "Estimated owners", "Release date", "Developers")
        .orderBy(F.col("Release date").desc())
        .limit(15)
)

display(df_top_owners)

There are not many games that crossed 50000000 users too. Among them we see popular shooter games like PUBG and CS-GO, but also relatively new games like Black Myth: Wukong and New World

Black Myth: Wukong is an action game based on chinese mythology 

New World - supernatural RPG



In [0]:
df_top_owners = (
    df_steam_fixed
        .filter(F.col("Estimated owners") == "20000000 - 50000000")
        .select("Name", "Estimated owners", "Genres", "Release date", "Developers")
        .orderBy(F.col("Release date").desc())
        .limit(30)
)

display(df_top_owners)

### Exploring games by reviews

In [0]:
from pyspark.sql import functions as F

top5_positive = (
    df_steam_fixed
        .orderBy(F.col("Positive").desc())
        .select("Name", "Positive", "Negative", "Developers", "Release date")
        .limit(5)
)

display(top5_positive)


Predictably, one of the most popular games get the most positive reviews. Here we see mostly online shooter games (CS-GO, Dota, PUBG), but also open-world GTA 5 and Terraria

In [0]:
top5_negative = (
    df_steam_fixed
        .orderBy(F.col("Negative").desc())
        .select("Name", "Positive", "Negative", "Developers", "Release date")
        .limit(5)
)

display(top5_negative)


Naturally, most popular games also get more negative reviews, so our top 5 is almost the same, except the Terraria, which was replaced by Tom Clancy's Rainbow Six® Siege - another obline RPG game

In [0]:
from pyspark.sql import functions as F

df_devs = df_steam_fixed.withColumn(
    "Developers_array",
    F.split(F.regexp_replace(F.col("Developers"), r"\s*,\s*", ","), ",")
)

df_devs = df_devs.withColumn(
    "Developers_array",
    F.expr("transform(Developers_array, x -> trim(x))")
)

df_dev_positive = (
    df_devs
        .withColumn("developer", F.explode("Developers_array"))
        .filter(
            (F.col("developer").isNotNull()) &
            (F.col("developer") != "") & (F.length("developer") > 4)
        )
        .groupBy("developer")
        .agg(F.sum("Positive").alias("total_positive"))
        .orderBy(F.col("total_positive").desc())
        .limit(5)
)
df_dev_negative = (
    df_devs
        .withColumn("developer", F.explode("Developers_array"))
        .filter(
            (F.col("developer").isNotNull()) &
            (F.col("developer") != "") & (F.length("developer") > 4)
        )
        .groupBy("developer")
        .agg(F.sum("Negative").alias("total_negative"))
        .orderBy(F.col("total_negative").desc())
        .limit(5)
)

display(df_dev_positive)


When it comes to developers and positive reviews, Valve is the absolute leader. 
* Valve - CS-GO and Half life developer
* Hidden Path Entertainment - CS-GO and Defence Grid (works with Valve)
* Ubisoft Montreal - Assasin's Creed, Far Cry
* Feral Interactive (Mac) - Hitman, Lara Croft games, Warhammer
* Facepunch Studios - creators of Rust, Garry's mod (works with Valve)

In [0]:
display(df_dev_negative)

Similarly to the case with games, most popular studios also have most of the negative reviews. Still, we've got new leader in negative reviews KRAFTON - creators of PUBG

### Top games by other criteria

In [0]:
#by Metacritic score
top5_metacritic = (
    df_steam_fixed
        .filter(F.col("Metacritic score").isNotNull())
        .orderBy(F.col("Metacritic score").desc())
        .select(
            "Name",
            "Metacritic score",
            "Estimated owners",
            "Release date",
            "Positive",
            "Negative"
        )
        .limit(10)
)

display(top5_metacritic)


Let's look into this top
* Disco Elysium - The Final Cut - detective role game 
* Persona 5 Royal - japanese role game
* Half-Life - action shooter
* Grand Theft Auto V - famous open-world game)
* BioShock™ - well known horror action shooter

In [0]:
# By number of achievements 

top5_achievements = (
    df_steam_fixed
        .orderBy(F.col("Achievements").desc())
        .select("Name", "Achievements", "Average playtime forever", "Release date")
        .limit(5)
)

display(top5_achievements)


In [0]:
# by Average playtime forever
top5_playtime = (
    df_steam_fixed
        .orderBy(F.col("Average playtime forever").desc())
        .select("Name", "Average playtime forever", "Price", "Genres", "Release date")
        .limit(5)
)

display(top5_playtime)


We got quite interesting top of games by the average playtime, so let's look into them
* Boom 3D - equalizer that makes game sounds 3D
* Energy Engine PC Live Wallpaper - live walpapers
* 副作用之瞳-Tlicolity Eyes- - Japanese visual novella, dating simulator
* Defense Clicker - free strategic game
* Relive - free game created to raise CPR averness and encourage users take CPR trainings

# BRONZE

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS steam_project.bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS steam_project.silver")
spark.sql("CREATE SCHEMA IF NOT EXISTS steam_project.gold")


In [0]:
df_bronze = (
    df_steam_fixed
    .withColumn("_ingest_ts", F.current_timestamp())
    .withColumn("_source", F.lit("steam_games_csv_v1"))
)

(
    df_bronze.write
    .format("delta")
    .option("delta.columnMapping.mode", "name") 
    .option("delta.minReaderVersion", "2")
    .option("delta.minWriterVersion", "5")
    .mode("overwrite")
    .saveAsTable("steam_project.bronze.steam_games_raw")
)


In [0]:
df_bronze_tbl = spark.table("steam_project.bronze.steam_games_raw")

print("Bronze sample")
df_bronze_tbl.show(5, truncate=False)
print("\nBronze schema")
df_bronze_tbl.printSchema()


rows_source       = df_steam_fixed.count()
rows_bronze_mem   = df_bronze.count()
rows_bronze_table = df_bronze_tbl.count()

print("\nRow count check")
print(f"df_steam_fixed   : {rows_source}")
print(f"df_bronze (mem)  : {rows_bronze_mem}")
print(f"bronze table     : {rows_bronze_table}")

assert rows_source == rows_bronze_mem,   "Mismatch between df_steam_fixed and df_bronze"
assert rows_source == rows_bronze_table, "Mismatch between df_steam_fixed and bronze table"

print("Row-count test passed: all row counts match.")

orig_cols   = set(df_steam_fixed.columns)
bronze_cols = set(df_bronze_tbl.columns) - {"_ingest_ts", "_source"}  # ignore metadata

missing_in_bronze = orig_cols - bronze_cols
extra_in_bronze   = bronze_cols - orig_cols

print("\nColumn presence check")
print("Missing in Bronze:", missing_in_bronze)
print("Extra in Bronze  :", extra_in_bronze)

assert not missing_in_bronze, "Some source columns are missing in Bronze!"
print("Column presence test passed: all source columns exist in Bronze.")


# SILVER

In [0]:
from pyspark.sql import functions as F
from functools import reduce
from operator import or_


df_bronze = spark.table("steam_project.bronze.steam_games_raw")
df_silver = df_bronze

NULL_PATTERNS = [
    r"^$",              # empty string
    r"^\s+$",           # whitespace
    r"(?i)^null$",      # null / NULL
    r"(?i)^none$",      # none / NONE
    r"(?i)^na$",        # na / NA
    r"(?i)^n/a$",       # n/a / N/A
    r"^\?$",            # ?
]

string_cols = [c for c, t in df_silver.dtypes if t == "string"]

for c in string_cols:
    cond = reduce(or_, [F.col(c).rlike(p) for p in NULL_PATTERNS])
    df_silver = df_silver.withColumn(
        c,
        F.when(cond, None).otherwise(F.col(c))
    )


array_cols_non_genres = [
    "Supported languages",
    "Full audio languages",
    "Categories",
    "Tags",
]

null_like_cond_expr = " OR ".join([f"x rlike '{p}'" for p in NULL_PATTERNS])

for c in array_cols_non_genres:
    if c not in df_silver.columns:
        continue

    df_silver = df_silver.withColumn(
        c,
        F.when(
            F.col(c).isNull(),
            None
        ).otherwise(
            F.expr(f"filter(`{c}`, x -> NOT ({null_like_cond_expr}))")
        )
    )

    df_silver = df_silver.withColumn(
        c,
        F.when(F.size(F.col(c)) == 0, None).otherwise(F.col(c))
    )


df_silver_wo_genres = df_silver.drop("Genres")

null_like_cond_g = reduce(
    or_,
    [F.col("g").rlike(p) for p in NULL_PATTERNS]
)

df_genres_clean = (
    df_bronze
    .select("AppID", "Genres")
    .withColumn("g", F.explode_outer("Genres"))
    .withColumn("g", F.regexp_replace("g", r"^\s+|\s+$", ""))
    .filter(~null_like_cond_g | F.col("g").isNull())
    .groupBy("AppID")
    .agg(
        F.collect_list("g").alias("Genres")
    )

    .withColumn(
        "Genres",
        F.when(
            (F.col("Genres").isNull()) | (F.size(F.col("Genres")) == 0),
            None
        ).otherwise(
            F.expr("filter(Genres, x -> x is not null and trim(x) != '')")
        )
    )
)

df_silver_final = (
    df_silver_wo_genres
    .join(df_genres_clean, on="AppID", how="left")
)

print("Sample Silver with fixed Genres:")
df_silver_final.select("AppID", "Genres").show(5, truncate=False)



(
    df_silver_final.write
    .format("delta")
    .option("delta.columnMapping.mode", "name")
    .option("delta.minReaderVersion", "2")
    .option("delta.minWriterVersion", "5")
    .mode("overwrite")
    .saveAsTable("steam_project.silver.steam_games_clean")
)

print("Silver overwritten with cleaned strings/arrays and fixed Genres")

In [0]:
from pyspark.sql import functions as F

df_silver = spark.table("steam_project.silver.steam_games_clean")

print("Silver non-null Genres:", df_silver.filter(F.col("Genres").isNotNull()).count())

df_silver.select(
    F.explode("Genres").alias("genre_raw")
).withColumn(
    "genre_trimmed", F.trim("genre_raw")
).groupBy("genre_trimmed").count().orderBy(F.desc("count")).show(20, truncate=False)


In [0]:
from functools import reduce
from operator import or_

NULL_PATTERNS = [
    r"^$",              
    r"^\s+$",           
    r"(?i)^null$",      
    r"(?i)^none$",      
    r"(?i)^na$",        
    r"(?i)^n/a$",       
    r"^\?$",            
]

array_cols = [
    "Supported languages",
    "Full audio languages",
    "Categories",
    "Genres",
    "Tags",
]

df_bronze = spark.table("steam_project.bronze.steam_games_raw")
df_silver_tbl = spark.table("steam_project.silver.steam_games_clean")

rows_bronze = df_bronze.count()
rows_silver = df_silver_tbl.count()
print(f"Rows Bronze: {rows_bronze}")
print(f"Rows Silver: {rows_silver}")
assert rows_bronze == rows_silver, "Row count mismatch between Bronze and Silver!"
print("Row-count test passed (Bronze == Silver)")

string_cols = [c for c, t in df_silver_tbl.dtypes if t == "string"]

cond_all = None
for c in string_cols:
    col_cond = reduce(or_, [F.col(c).rlike(p) for p in NULL_PATTERNS])
    cond_all = col_cond if cond_all is None else (cond_all | col_cond)

remaining_str = df_silver_tbl.filter(cond_all).count() if cond_all is not None else 0
print("Remaining null-like strings:", remaining_str)
assert remaining_str == 0, "There are still null-like strings in Silver!"
print("All null-like string values removed in Silver")

array_cols_present = [c for c in array_cols if c in df_silver_tbl.columns]
if array_cols_present:
    empty_arrays = df_silver_tbl.filter(
        reduce(or_, [F.size(F.col(c)) == 0 for c in array_cols_present])
    ).count()
else:
    empty_arrays = 0

print("Empty arrays remaining:", empty_arrays)
assert empty_arrays == 0, "Some empty arrays remain in Silver!"
print("No empty arrays left in Silver")

print("Silver cleaning complete and tests passed.")


# GOLD

In [0]:
from pyspark.sql import functions as F

df_silver = spark.table("steam_project.silver.steam_games_clean")

print("Rows in Silver:", df_silver.count())

df_price_stats_gold = (
    df_silver
        .withColumn("year", F.year("Release date"))
        .filter(
            (F.col("Release date").isNotNull()) &
            (F.col("year").between(2010, 2024)) &
            (F.col("Price").isNotNull()) &
            (F.col("Price") > 0)
        )
        .groupBy("year")
        .agg(
            F.mean("Price").alias("avg_price"),
            F.expr("percentile_approx(Price, 0.5)").alias("median_price"),
            F.min("Price").alias("min_price"),
            F.max("Price").alias("max_price"),
            F.count("*").alias("num_games")
        )
        .orderBy("year")
)

(
    df_price_stats_gold.write
    .format("delta")
    .option("delta.columnMapping.mode", "name")
    .option("delta.minReaderVersion", "2")
    .option("delta.minWriterVersion", "5")
    .mode("overwrite")
    .saveAsTable("steam_project.gold.games_price_stats_by_year")
)

print("GOLD 1 written: steam_project.gold.games_price_stats_by_year")

g1 = spark.table("steam_project.gold.games_price_stats_by_year")
print("GOLD 1 rows:", g1.count())
g1.show(5)

min_year = g1.select(F.min("year")).first()[0]
max_year = g1.select(F.max("year")).first()[0]
assert 2010 <= min_year <= 2024, "Min year in GOLD 1 out of expected range"
assert 2010 <= max_year <= 2024, "Max year in GOLD 1 out of expected range"
print("GOLD 1 tests passed")

target_genres = ["Action", "Indie", "Casual"]

df_genre_trends_gold = (
    df_silver
        .withColumn("year", F.year("Release date"))
        .filter(F.col("year").isNotNull())
        .withColumn("genre_raw", F.explode("Genres"))      # explode array
        .withColumn("genre", F.trim(F.col("genre_raw")))   # trim spaces
        .filter(F.col("genre").isin(target_genres))
        .groupBy("year", "genre")
        .count()
        .orderBy("year", "genre")
)

(
    df_genre_trends_gold.write
    .format("delta")
    .option("delta.columnMapping.mode", "name")
    .option("delta.minReaderVersion", "2")
    .option("delta.minWriterVersion", "5")
    .mode("overwrite")
    .saveAsTable("steam_project.gold.games_genre_trends_key_genres")
)

print("GOLD 2 written: steam_project.gold.games_genre_trends_key_genres")

g2 = spark.table("steam_project.gold.games_genre_trends_key_genres")
print("GOLD 2 rows:", g2.count())
g2.show(20)

genres_in_table = [r[0] for r in g2.select("genre").distinct().collect()]
unexpected_genres = set(genres_in_table) - set(target_genres)
assert not unexpected_genres, f"GOLD 2 contains unexpected genres: {unexpected_genres}"
print("GOLD 2 tests passed")

df_fp_year = (
    df_silver
        .withColumn("year", F.year("Release date"))
        .filter(F.col("year").isNotNull())
        .withColumn("type", F.when(F.col("Price") == 0, "Free").otherwise("Paid"))
        .groupBy("year", "type")
        .count()
)

df_fp_share_gold = (
    df_fp_year
        .groupBy("year")
        .pivot("type", ["Free", "Paid"])
        .sum("count")
        .fillna(0)
        .withColumn("total", F.col("Free") + F.col("Paid"))
        .withColumn("pct_free", F.round(F.col("Free") / F.col("total"), 3))
        .withColumn("pct_paid", F.round(F.col("Paid") / F.col("total"), 3))
        .orderBy("year")
)

(
    df_fp_share_gold.write
    .format("delta")
    .option("delta.columnMapping.mode", "name")
    .option("delta.minReaderVersion", "2")
    .option("delta.minWriterVersion", "5")
    .mode("overwrite")
    .saveAsTable("steam_project.gold.games_free_vs_paid_share")
)

print("GOLD 3 written: steam_project.gold.games_free_vs_paid_share")

g3 = spark.table("steam_project.gold.games_free_vs_paid_share")
print("GOLD 3 rows:", g3.count())
g3.show(5)

max_diff = g3.select(F.max(F.abs(F.col("pct_free") + F.col("pct_paid") - 1))).first()[0]
assert max_diff < 1e-6, "pct_free + pct_paid != 1 for some years in GOLD 3"
print("GOLD 3 tests passed")

df_quarter_gold = (
    df_silver
        .withColumn("year", F.year("Release date"))
        .withColumn("quarter", F.quarter("Release date"))
        .filter(F.col("year").isNotNull() & F.col("quarter").isNotNull())
        .groupBy("year", "quarter")
        .count()
        .orderBy("year", "quarter")
)

(
    df_quarter_gold.write
    .format("delta")
    .option("delta.columnMapping.mode", "name")
    .option("delta.minReaderVersion", "2")
    .option("delta.minWriterVersion", "5")
    .mode("overwrite")
    .saveAsTable("steam_project.gold.games_releases_by_quarter")
)

print("GOLD 4 written: steam_project.gold.games_releases_by_quarter")

g4 = spark.table("steam_project.gold.games_releases_by_quarter")
print("GOLD 4 rows:", g4.count())
g4.show(5)

quarters = [r[0] for r in g4.select("quarter").distinct().collect()]
assert set(quarters).issubset({1, 2, 3, 4}), f"Unexpected quarter values in GOLD 4: {quarters}"
print("GOLD 4 tests passed")

print("All GOLD tables created and validated.")


In [0]:
from pyspark.sql import functions as F

df_bronze = spark.table("steam_project.bronze.steam_games_raw")

df_bronze_1997 = (
    df_bronze
        .filter(F.year(F.col("Release date")) == 1997)
)

display(df_bronze_1997)
